In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

start = "2024-11-1"

# NASDAQ
NAS = yf.download('^IXIC', interval="1d", start=start)[['Adj Close']].pct_change()
# Dow Jones Industrial Average
DJI = yf.download('^DJI', interval="1d", start=start)[['Adj Close']].pct_change()
# S&P 500 (as the benchmark)
SP5 = yf.download('^GSPC', interval="1d", start=start)[['Adj Close']].pct_change()

NAS.columns, DJI.columns, SP5.columns = ['NAS'], ['DJI'], ['SP5']
data = pd.concat([NAS, DJI, SP5], axis=1).dropna()
data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,NAS,DJI,SP5
Date,,,
2024-11-04 00:00:00+00:00,-0.003286,-0.006125,-0.002812
2024-11-05 00:00:00+00:00,0.014257,0.010223,0.012266
2024-11-06 00:00:00+00:00,0.029519,0.035717,0.025296
2024-11-07 00:00:00+00:00,0.015065,-0.000013,0.007431
2024-11-08 00:00:00+00:00,0.000899,0.005938,0.003757
2024-11-11 00:00:00+00:00,0.000621,0.006914,0.000969
2024-11-12 00:00:00+00:00,-0.000900,-0.008628,-0.002893
2024-11-13 00:00:00+00:00,-0.002628,0.001075,0.000232
2024-11-14 00:00:00+00:00,-0.006400,-0.004717,-0.006050


In [ ]:
res = sm.OLS(data.NAS, sm.add_constant(data.SP5)).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    NAS   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     137.7
Date:                Thu, 05 Dec 2024   Prob (F-statistic):           2.02e-10
Time:                        02:37:08   Log-Likelihood:                 93.064
No. Observations:                  22   AIC:                            -182.1
Df Residuals:                      20   BIC:                            -179.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.421e-05      0.001      0.040      0.968      -0.002       0.002
SP5            1.2948      0.110     11.734      0.000       1.065       1.525
==============================================================================
Omnibus:                        0.637   Durbin-Watson:                   1.772
Prob(Omnibus):                  0.727   Jarque-Bera (JB):                0.675
Skew:                           0.183   Prob(JB):                        0.714
Kurtosis:                       2.224   Cond. No.                         140.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
dir(res)
res.params
res.params.iloc[1]
res.resid.std()

1.9400435572059314e-18

In [ ]:
benchmark = 'SP5'
RBenchmark = data[benchmark]

perfEval = pd.DataFrame(index=['AriRet', 'Sharpe', 'Beta', 'TreyRatio',
                               'JenAlpha',  'AppRatio', 'M2', 'IndAlpha',
                               'TE', 'InfoRatio'],
                        columns=['NAS'])
# about 252 trading days per year
TDays = 252

# annualized the Sharpe Ratio from daily data
SharpeBenchmark = RBenchmark.mean()/RBenchmark.std() * (TDays)**0.5

VolaBenchmark = RBenchmark.std() * (TDays)**0.5

for p in ['NAS', 'DJI', 'SP5']:
    RP = data[p]

    perfEval.loc['AriRet', p] = RP.mean() * TDays
    perfEval.loc['Sharpe', p] = RP.mean()/RP.std() * (TDays)**0.5

    res = sm.OLS(RP, sm.add_constant(RBenchmark)).fit()

    perfEval.loc['Beta', p]      = res.params.loc[benchmark]
    perfEval.loc['TreyRatio', p] = RP.mean()/res.params.loc[benchmark] * TDays
    perfEval.loc['JenAlpha', p]  = res.params.iloc[0] * TDays
    perfEval.loc['AppRatio', p]  = round(res.params.iloc[0], 10)/res.resid.std() * ((TDays)**0.5)
    perfEval.loc['M2', p] = (perfEval.loc['Sharpe', p] - SharpeBenchmark) * VolaBenchmark
    perfEval.loc['IndAlpha', p]  = (RP - RBenchmark).mean() * TDays
    perfEval.loc['TE', p]        = (RP - RBenchmark).std() * ((TDays)**0.5)
    perfEval.loc['InfoRatio', p] = perfEval.loc['IndAlpha', p]/perfEval.loc['TE', p]

display(np.round(perfEval.astype(float), 3))

<ipython-input-12-f4f9ae92e72d>:31: RuntimeWarning: invalid value encountered in scalar divide
  perfEval.loc['InfoRatio', p] = perfEval.loc['IndAlpha', p]/perfEval.loc['TE', p]


,NAS,DJI,SP5
AriRet,0.916,0.791,0.701
Sharpe,5.705,5.310,6.048
Beta,1.295,1.088,1.000
TreyRatio,0.708,0.727,0.701
JenAlpha,0.009,0.028,0.000
AppRatio,0.151,0.357,0.000
M2,-0.040,-0.086,0.000
IndAlpha,0.215,0.090,0.000
TE,0.067,0.080,0.000
InfoRatio,3.231,1.128,NaN


In [ ]:
MktTime = data[['SP5']].copy()

MktTime['SqRm'] = MktTime['SP5']**2

for p in ['NAS', 'DJI',]:
    RP = data[p]

    res = sm.OLS(RP, sm.add_constant(MktTime)).fit()

    print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    NAS   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     76.26
Date:                Thu, 05 Dec 2024   Prob (F-statistic):           8.36e-10
Time:                        02:49:48   Log-Likelihood:                 94.553
No. Observations:                  22   AIC:                            -183.1
Df Residuals:                      19   BIC:                            -179.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.001      0.413      0.6

In [ ]:
from google.colab import files

data.to_csv('data.csv')
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
MktTime

,SP5,SqRm
Date,,
2024-11-04 00:00:00+00:00,-0.002812,7.907813e-06
2024-11-05 00:00:00+00:00,0.012266,1.504460e-04
2024-11-06 00:00:00+00:00,0.025296,6.398840e-04
2024-11-07 00:00:00+00:00,0.007431,5.522318e-05
2024-11-08 00:00:00+00:00,0.003757,1.411380e-05
2024-11-11 00:00:00+00:00,0.000969,9.390839e-07
2024-11-12 00:00:00+00:00,-0.002893,8.367480e-06
2024-11-13 00:00:00+00:00,0.000232,5.392972e-08
2024-11-14 00:00:00+00:00,-0.006050,3.659929e-05
